In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

import cv2
from PIL import Image
import pytesseract
import re
import numpy as np


from pyzbar.pyzbar import decode

import requests
import json

In [2]:
# Scrapping de la web e-aditivos.com para sacar la info de los aditivos.

def get_table_additives():
    URL = "https://e-aditivos.com/"

    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table")
    output_rows = []
    for table_row in table.findAll('tr'):
        columns = table_row.findAll('td')
        output_row = []
        for column in columns:
            output_row.append(column.text)
            output_rows.append(output_row)
    additives= pd.DataFrame(output_rows,columns=['id','tipo','origen','aditivo','clasificación'])[1:]
    additives_obj = additives.select_dtypes(['object'])
    additives[additives_obj.columns] = additives_obj.apply(lambda item: item.str.strip())
    allergic_e ='E-1404','E-1410','E-1412','E-1413','E-1414','E-1420','E-1422','E1440','E-1442','E-1450','E-4511','E-4512','E-4513','E-327','E-106','E-101','E-270','E-325','E-326','E-472','E-478','E-480','E-482','E-481','E-966','E-150','E-260','E-330','E-300',
    additives['aditivos_alergicos'] = np.where(additives['id'].isin(allergic_e), "Warning, this could kill you!", 'Dont worry, be happy!')
    additives['id'].replace('E-','E', regex=True, inplace=True)
    additives.drop_duplicates(keep='first',inplace=True)
    return additives

In [3]:
additives = get_table_additives()

In [4]:
additives

,id,tipo,origen,aditivo,clasificación,aditivos_alergicos
1,E100,No nocivo,Vegetal,Curcumina,Colorante Colorantes,"Dont worry, be happy!"
5,E101,No nocivo,Animal,Riboflavina,Colorante Colorantes,"Warning, this could kill you!"
10,E102,¡Peligroso!,Químico o sintético,"Tartracina, Tartrazina, Tartrazine, Amarillo 5...",Colorante Colorantes,"Dont worry, be happy!"
15,E104,¡Peligroso!,Químico o sintético,"Amarillo quinoleina, quinoline yellow",Colorante Colorantes,"Dont worry, be happy!"
20,E106,No nocivo,Indeterminado,Lactoflavina,Colorante Colorantes,"Warning, this could kill you!"
...,...,...,...,...,...,...
1330,E1422,Sospechoso,Químico o sintético,Adipato dialmidón acetilado,Estabilizante Estabilizantes,"Warning, this could kill you!"
1335,E1440,Sospechoso,Químico o sintético,Hidroxipropil almidón,Estabilizante Estabilizantes,"Dont worry, be happy!"
1340,E1442,Sospechoso,Químico o sintético,Fosfato dialmidón hidroxipropilado,Estabilizante Estabilizantes,"Warning, this could kill you!"
1345,E1450,Sospechoso,Químico o sintético,Octenil succinato sódico de almidón,Estabilizante Estabilizantes,"Warning, this could kill you!"


In [6]:
def capture_food():
    url = 'http://192.168.20.122:8080/video'
    cap = cv2.VideoCapture(url)
    while(True):
        ret, frame = cap.read()
        if frame is not None:
            cv2.imshow('frame',frame)
            cv2.imwrite(filename='saved_img.jpg', img=frame)
        q = cv2.waitKey(1)
        if q == ord("q"):
            break
    cv2.destroyAllWindows()
    im = Image.open("saved_img.jpg")

    text = pytesseract.image_to_string(im, lang = 'spa')
    return text

#  tengo que cambiarlo y probar lo que vi con JLuna

In [7]:
text = capture_food()

In [8]:
text

"Tortilla Yorl:\n\nYork ham and potato omele'ta\nINGREDIENTES: PAN DE MOLDE 41% (harina qe\nagua, levadura, aceite corclin ia os) R lo,\n\nA dol\n\nE Es) q 7%\n(patata , Íqu euizado, ace o\nl cebolla, aceite de oliva, sa!\ne corrector crece y Sal), FAC O\nwa, almidón, ar\nAN\n(E-120), e (E-2),\nvegetales, agua,"

In [9]:
def find_text(text):
    result = re.findall(r'E-\d+', text)
    result_2 = re.findall(r'E\d+', text)
    return result, result_2

In [10]:
def get_homogenic_data(result,result_2):
    values = []
    for value in result:
        values.append(value)
    for value in result_2:
        values.append(value)
    for value in values:
        if "E-" in value:
            values = [item.replace('E-','E') for item in values]
        else:
            values
    return values

In [11]:
result, result_2 = find_text(text)

In [12]:
values = get_homogenic_data(result,result_2)

In [13]:
def get_allergies_dataframe(values):
    allergies = additives[additives['id'].isin(values)]
    return allergies

In [14]:
allergies = get_allergies_dataframe(values)

In [15]:
allergies

,id,tipo,origen,aditivo,clasificación,aditivos_alergicos
30,E120,¡Peligroso!,Animal,Cochinilla,Colorante Colorantes,"Dont worry, be happy!"


In [ ]:
#  Reconocimiento de codigo de barras para su lectura.

def capture_barcode():
    url = 'http://192.168.0.154:8080/video'
    cap = cv2.VideoCapture(url)
    lastCode = ""

    while True:

        _, frame = cap.read()


        frame = cv2.flip(frame, 1) 


        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 

        barcodes = decode(gray)    


        for barcode in barcodes:

            (x,y,w,h) = barcode.rect

            cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)

            barcodeType = barcode.type
            barcodeData = barcode.data.decode("utf-8")

            dataText = f"Data: {barcodeData}"
            dataType = f"Type: {barcodeType}"
            cv2.putText(frame, dataText, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
            cv2.putText(frame, dataType, (x,y-30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)


            if barcodeData != lastCode:
                lastCode = barcodeData
                #print("READ: " + barcodeData)

        cv2.imshow("original", frame)


        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return barcodeData

In [ ]:
barcodeData = capture_barcode()

In [ ]:
def nutri_score():
    URL = "https://world.openfoodfacts.org/api/v0/product/" + barcodeData +".json"

    response = requests.get(URL)

    contenant = response.content
    res = json.loads(contenant)
    nutri_score = res.get('product').get('nutrition_grades')

    nutri_score = nutri_score.upper()
    return res.get('product').get('nutrient_levels'), "Nutri-Score:" + " " + nutri_score

In [ ]:
nutri = nutri_score()

In [ ]:
nutri